# Portfolio recommendation

In [1]:
import numpy as np
import pandas as pd

# Generate synthetic stock price data
np.random.seed(42)
dates = pd.date_range('2021-01-01', periods=1000)
stock_prices = pd.DataFrame({
    'Stock_A': np.random.normal(100, 10, 1000).cumprod(),
    'Stock_B': np.random.normal(100, 15, 1000).cumprod(),
    'Stock_C': np.random.normal(100, 20, 1000).cumprod(),
    'Stock_D': np.random.normal(100, 5, 1000).cumprod()
}, index=dates)

print(stock_prices.head())


                 Stock_A       Stock_B       Stock_C       Stock_D
2021-01-01  1.049671e+02  1.209903e+02  8.649643e+01  9.046096e+01
2021-01-02  1.035158e+04  1.377711e+04  8.399636e+03  8.656940e+03
2021-01-03  1.102204e+06  1.390034e+06  7.068429e+05  8.477912e+05
2021-01-04  1.270073e+08  1.255144e+08  6.633068e+07  9.278095e+07
2021-01-05  1.240334e+10  1.386600e+10  4.120973e+09  9.536282e+09


C:\Users\diptiwar\AppData\Local\Temp\ipykernel_19352\583696485.py:8: RuntimeWarning: overflow encountered in accumulate
  'Stock_A': np.random.normal(100, 10, 1000).cumprod(),
C:\Users\diptiwar\AppData\Local\Temp\ipykernel_19352\583696485.py:9: RuntimeWarning: overflow encountered in accumulate
  'Stock_B': np.random.normal(100, 15, 1000).cumprod(),
C:\Users\diptiwar\AppData\Local\Temp\ipykernel_19352\583696485.py:10: RuntimeWarning: overflow encountered in accumulate
  'Stock_C': np.random.normal(100, 20, 1000).cumprod(),
C:\Users\diptiwar\AppData\Local\Temp\ipykernel_19352\583696485.py:11: RuntimeWarning: overflow encountered in accumulate
  'Stock_D': np.random.normal(100, 5, 1000).cumprod()


In [5]:
#Modeling and Forecasting
from statsmodels.tsa.arima.model import ARIMA
from arch import arch_model

# Calculate returns
returns = stock_prices.pct_change().dropna()

# Forecast returns using ARIMA
model_arima = ARIMA(returns['Stock_A'], order=(1,1,1))
fit_arima = model_arima.fit()
forecast_arima = fit_arima.forecast(steps=10)

# Forecast volatility using GARCH
model_garch = arch_model(forecast_arima, vol='Garch', p=1, q=1)
fit_garch = model_garch.fit()
forecast_garch = fit_garch.forecast(horizon=10)

print("ARIMA Forecast Returns:", forecast_arima)
print("GARCH Forecast Volatility:", forecast_garch.variance.iloc[-1])


Iteration:      1,   Func. Count:      6,   Neg. LLF: 1646413471.7725172
Iteration:      2,   Func. Count:     15,   Neg. LLF: 4609397.826754228
Iteration:      3,   Func. Count:     21,   Neg. LLF: -18.756067577815163
Iteration:      4,   Func. Count:     26,   Neg. LLF: -15.849826629057782
Iteration:      5,   Func. Count:     38,   Neg. LLF: 6991288.85198286
Iteration:      6,   Func. Count:     52,   Neg. LLF: 68165.47297552094
Iteration:      7,   Func. Count:     67,   Neg. LLF: 16291306.955013836
Iteration:      8,   Func. Count:     82,   Neg. LLF: 311770.16179259826
Iteration:      9,   Func. Count:     92,   Neg. LLF: 9623081872311.678
Iteration:     10,   Func. Count:    107,   Neg. LLF: -9.512795016922173
Iteration:     11,   Func. Count:    114,   Neg. LLF: 12526482.31778598
Iteration:     12,   Func. Count:    128,   Neg. LLF: 396575.15191174234
Iteration:     13,   Func. Count:    140,   Neg. LLF: 46264.91909984403
Iteration:     14,   Func. Count:    151,   Neg. LLF: 45

C:\Users\diptiwar\AppData\Roaming\Python\Python311\site-packages\arch\univariate\base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.004902. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(


In [18]:
import numpy as np
import pandas as pd
import yfinance as yf

# Download historical price data for 4 stocks
tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN']
data = yf.download(tickers, start='2021-01-01', end='2023-01-01')['Adj Close']

# Calculate daily returns
returns = data.pct_change().dropna()

print(returns.head())


[*********************100%%**********************]  4 of 4 completed

Ticker          AAPL      AMZN     GOOGL      MSFT
Date                                              
2021-01-05  0.012364  0.010004  0.008064  0.000965
2021-01-06 -0.033661 -0.024897 -0.009868 -0.025929
2021-01-07  0.034123  0.007577  0.029869  0.028457
2021-01-08  0.008631  0.006496  0.013239  0.006093
2021-01-11 -0.023249 -0.021519 -0.023106 -0.009699


In [19]:
from arch import arch_model

# Function to fit GARCH(1,1) model and forecast volatility
def garch_volatility_forecast(returns, horizon=5):
    model = arch_model(returns, vol='Garch', p=1, q=1)
    model_fit = model.fit(disp='off')
    forecast = model_fit.forecast(horizon=horizon)
    return forecast.variance.values[-1, :]

# Forecast volatility for each stock
vol_forecasts = {}
for ticker in tickers:
    vol_forecasts[ticker] = garch_volatility_forecast(returns[ticker])

print("Volatility Forecasts:", vol_forecasts)


C:\Users\diptiwar\AppData\Roaming\Python\Python311\site-packages\arch\univariate\base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0003769. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(


Volatility Forecasts: {'AAPL': array([0.00045653, 0.00045494, 0.00045338, 0.00045185, 0.00045035]), 'MSFT': array([0.00037116, 0.00037044, 0.00036974, 0.00036906, 0.00036839]), 'GOOGL': array([0.00044694, 0.00044633, 0.00044573, 0.00044514, 0.00044456]), 'AMZN': array([0.00051494, 0.00051713, 0.00051928, 0.00052138, 0.00052344])}


C:\Users\diptiwar\AppData\Roaming\Python\Python311\site-packages\arch\univariate\base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0003354. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
C:\Users\diptiwar\AppData\Roaming\Python\Python311\site-packages\arch\univariate\base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0004157. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
C:\Users\diptiwar\AppData\Roamin

In [20]:
import scipy.stats as stats

def calculate_var_cvar(returns, alpha=0.05):
    var = np.percentile(returns, alpha * 100)
    cvar = returns[returns <= var].mean()
    return var, cvar

# Calculate VaR and CVaR for each stock
var_cvar = {}
for ticker in tickers:
    var_cvar[ticker] = calculate_var_cvar(returns[ticker])

print("VaR and CVaR:", var_cvar)


VaR and CVaR: {'AAPL': (-0.03318978404436427, -0.040847471661977504), 'MSFT': (-0.029351469951767295, -0.04030802750001284), 'GOOGL': (-0.033006954431352334, -0.043965233190519766), 'AMZN': (-0.03698730142215295, -0.05756125998012648)}


In [21]:
import numpy as np
import cvxpy as cp

# Expected returns (mean returns)
expected_returns = returns.mean()

# Covariance matrix
cov_matrix = returns.cov()

# Number of stocks
n = len(tickers)

# Define variables
weights = cp.Variable(n)
returns_portfolio = expected_returns.T @ weights
risk_portfolio = cp.quad_form(weights, cov_matrix)

# Define problem (maximize Sharpe ratio)
risk_free_rate = 0.01
objective = cp.Maximize((returns_portfolio - risk_free_rate) / cp.sqrt(risk_portfolio))
constraints = [cp.sum(weights) == 1, weights >= 0]
problem = cp.Problem(objective, constraints)

# Solve the problem
problem.solve()
optimal_weights = weights.value

print("Optimal Weights:", {tickers[i]: optimal_weights[i] for i in range(n)})


IndexError: tuple index out of range

In [22]:
# Define variables
import cvxpy as cp
weights = cp.Variable(4)
weights

Variable((4,), var4)

In [16]:
# Expected returns (mean returns)
expected_returns = returns.mean()

In [17]:
returns_portfolio = expected_returns.T @ weights

IndexError: tuple index out of range

In [23]:
import numpy as np
import pandas as pd
import yfinance as yf

# Download historical price data for the stocks
tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN']
data = yf.download(tickers, start='2021-01-01', end='2022-01-01')['Adj Close']

# Calculate daily returns and statistics
returns = data.pct_change().dropna()
expected_returns = returns.mean() * 250  # Annualized
cov_matrix = returns.cov() * 250  # Annualized
risk_free_rate = 0.03  # Assuming a 3% risk-free rate

# Initialize lists to store portfolio returns, volatilities, and Sharpe ratios
pfolio_returns = []
pfolio_volatilities = []
sharpe_ratios = []
weights_list = []

num_assets = len(tickers)
num_portfolios = 1000

# Monte Carlo simulation to generate random portfolios
for i in range(num_portfolios):
    weights = np.random.random(num_assets)
    weights /= np.sum(weights)
    portfolio_return = np.sum(weights * expected_returns)
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_volatility
    
    pfolio_returns.append(portfolio_return)
    pfolio_volatilities.append(portfolio_volatility)
    sharpe_ratios.append(sharpe_ratio)
    weights_list.append(weights)

# Convert lists to numpy arrays
pfolio_returns = np.array(pfolio_returns)
pfolio_volatilities = np.array(pfolio_volatilities)
sharpe_ratios = np.array(sharpe_ratios)
weights_list = np.array(weights_list)

# Identify the portfolio with the highest Sharpe ratio
max_sharpe_idx = sharpe_ratios.argmax()
optimal_weights = weights_list[max_sharpe_idx]

# Display the results
print("Optimal Weights (Tangency Portfolio):")
for ticker, weight in zip(tickers, optimal_weights):
    print(f"{ticker}: {weight:.4f}")

print(f"\nPortfolio Return: {pfolio_returns[max_sharpe_idx]:.2%}")
print(f"Portfolio Volatility: {pfolio_volatilities[max_sharpe_idx]:.2%}")
print(f"Sharpe Ratio: {sharpe_ratios[max_sharpe_idx]:.2f}")


[*********************100%%**********************]  4 of 4 completed


Optimal Weights (Tangency Portfolio):
AAPL: 0.0352
MSFT: 0.0222
GOOGL: 0.3969
AMZN: 0.5457

Portfolio Return: 48.35%
Portfolio Volatility: 20.43%
Sharpe Ratio: 2.22
